In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import zipfile
zf = zipfile.ZipFile('/content/drive/MyDrive/COMP723Data/CovidNewArticles.zip')
test_covid = pd.read_csv(zf.open('Corona_NLP_test.csv'))
train_covid = pd.read_csv(zf.open('Corona_NLP_train.csv'), encoding='ISO-8859-1')


In [2]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [3]:
# Training data
label_mapping = {'Positive': 'pos', 'Negative': 'neg', 'Neutral': 'neu', 'Extremely Positive': 'pos', 'Extremely Negative': 'neg'}

# Replace the labels using the dictionary
train_covid['Sentiment'] = train_covid['Sentiment'].replace(label_mapping)
test_covid['Sentiment'] = test_covid['Sentiment'].replace(label_mapping)


X_train = train_covid["OriginalTweet"]
y_train = train_covid["Sentiment"]

# Testing data
X_test = test_covid["OriginalTweet"]
y_test = test_covid["Sentiment"]




In [4]:
# Initialize a TF-IDF vectorizer to convert text data into numerical features
tfidf_vectorizer = TfidfVectorizer(max_features=4000)  # You can adjust the number of features as needed

# Transform the training data into TF-IDF features
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform the testing data into TF-IDF features
X_test_tfidf = tfidf_vectorizer.transform(X_test)




In [5]:

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


#Create a Naive Bayes classifier
classifier = MultinomialNB()

# Define hyperparameters to search
param_grid = {
    'alpha': [0.1, 1.0, 10.0],  # Smoothing parameter
}

# Create a GridSearchCV object
grid_search1 = GridSearchCV(classifier, param_grid, cv=5, scoring='accuracy')

# Fit the grid search to your data
grid_search1.fit(X_train_tfidf, y_train)

# Get the best hyperparameters
best_params1 = grid_search1.best_params_

best_alpha = best_params1['alpha']

nb_classifier = MultinomialNB(alpha=best_alpha)

# Fit the classifier to your entire training dataset
nb_classifier.fit(X_train_tfidf, y_train)
#LogisticRegression
logistic = LogisticRegression()
decision = DecisionTreeClassifier()
forest = RandomForestClassifier()
svm = SVC()


# # Train the classifier on the training data

classifier.fit(X_train_tfidf, y_train)
logistic.fit(X_train_tfidf, y_train)
decision.fit(X_train_tfidf, y_train)
forest.fit(X_train_tfidf, y_train)
svm.fit(X_train_tfidf, y_train)



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


SVC()

In [6]:
from sklearn.metrics import accuracy_score, classification_report

# # Predict on the test data
y_pred = nb_classifier.predict(X_test_tfidf)
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Naive Bayes:", accuracy)

# l_pred = linearR.predict(X_test)
# # Calculate accuracy
# accuracy = accuracy_score(y_test, l_pred)
# print("Linear Regression:", accuracy)

logistic_pred = logistic.predict(X_test_tfidf)
# Calculate accuracy
accuracy = accuracy_score(y_test, logistic_pred)
print("Logistic Regression:", accuracy)

d_pred = decision.predict(X_test_tfidf)
# Calculate accuracy
accuracy = accuracy_score(y_test, d_pred)
print("Decision Tree:", accuracy)

f_pred = forest.predict(X_test_tfidf)
# Calculate accuracy
accuracy = accuracy_score(y_test, f_pred)
print("Random Forest:", accuracy)

svm_pred = svm.predict(X_test_tfidf)
# Calculate accuracy
accuracy = accuracy_score(y_test, svm_pred)
print("Svm:", accuracy)

# Generate a classification report
# print("Naive Bayes Report:")
# print(classification_report(y_test, y_pred))

# print("Logistic Regression Report:")
# print(classification_report(y_test, logistic_pred))

# print("Decision Tree Report:")
# print(classification_report(y_test, d_pred))

# print("Random Forest Report:")
# print(classification_report(y_test, f_pred))

# print("SVM Report:")
# print(classification_report(y_test, svm_pred))


Naive Bayes: 0.6837809373354397
Logistic Regression: 0.7991047919957872
Decision Tree: 0.5747761979989469
Random Forest: 0.667983149025803
Svm: 0.7877830437072143


In [8]:
new_text = "hate it"
new_text_features = tfidf_vectorizer.transform([new_text])

#Bayes
predicted_sentiment = nb_classifier.predict(new_text_features)[0]
print("Predicted Sentiment Bayes:", predicted_sentiment)
#logistic
predicted_sentiment = logistic.predict(new_text_features)[0]
print("Predicted Sentiment logistic:", predicted_sentiment)
#Decision treee
predicted_sentiment = decision.predict(new_text_features)[0]
print("Predicted Sentiment Decision tree:", predicted_sentiment)
#Forest
predicted_sentiment = forest.predict(new_text_features)[0]
print("Predicted Sentiment forest:", predicted_sentiment)

predicted_sentiment = svm.predict(new_text_features)[0]
print("Predicted Sentiment Svm:", predicted_sentiment)

Predicted Sentiment Bayes: neg
Predicted Sentiment logistic: neg
Predicted Sentiment Decision tree: neu
Predicted Sentiment forest: neu
Predicted Sentiment Svm: neg


In [9]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import RandomOverSampler
import random

df1 = pd.read_csv('/content/drive/MyDrive/Tower_Sentiment.csv', nrows=10000)
df2 = pd.read_csv('/content/drive/MyDrive/Tuatahi_Sentiment_CSV.csv', nrows=10000)
df3 = pd.read_csv('/content/drive/MyDrive/Laybuy_Sentiment.csv', nrows=10000)
df4 = pd.read_csv('/content/drive/MyDrive/T7_Sentiment_CSV.csv', encoding='latin1', nrows=10000)
df5 = pd.read_csv('/content/drive/MyDrive/NZPost_Sentiment.csv', nrows=10000)

def dropcolumns(df):
    columns_to_keep = ['Text', 'ID', 'Sentiment', 'FullSentiment', 'Source']  # Replace with the columns you want to keep

    # Drop all columns except the ones specified in columns_to_keep
    columns_to_drop = [col for col in df.columns if col not in columns_to_keep]
    df.drop(columns=columns_to_drop, inplace=True)
    df = df[~df["Source"].str.contains("bot")]
    df = df.drop(columns=["Source"])
    return df

# Dropping columns that do not need to be read

# Truncate the DataFrame to contain only the first 3007 rows



# Dropping columns that do not need to be read
df1 = dropcolumns(df1)
df2 = dropcolumns(df2)
df3 = dropcolumns(df3)
df4 = dropcolumns(df4)
df5 = dropcolumns(df5)

In [10]:


def cleandata(data):
  #Current amount of rows in dataset
  print(len(data))
  #dropping rows after 1500 because thats where we annotated to
  #df.drop(df.index[1500:], inplace=True)
  print(len(data))
  new = data['Text'].fillna('')
  grouped_df = data.groupby('ID').agg({'Text': lambda x: '. '.join(x.astype(str)), 'FullSentiment': 'first'}).reset_index()

  data.drop(data.columns[[1,3]], axis=1, inplace=True)
  #Dropping all NaN data points
  df_filled = data.dropna()
  #df_filled = df.fillna(" ")
  old_word = 'Neural'
  new_word = 'neu'
  # Find rows containing the target word and replace it with the new word
  df_filled["Sentiment"] = df_filled["Sentiment"].replace(old_word, new_word)

  old_word = 'Positve'
  new_word = 'pos'
  # Find rows containing the target word and replace it with the new word
  df_filled["Sentiment"] = df_filled["Sentiment"].replace(old_word, new_word)

  old_word = 'positive'
  new_word = 'pos'
  # Find rows containing the target word and replace it with the new word
  df_filled["Sentiment"] = df_filled["Sentiment"].replace(old_word, new_word)

  old_word = 'Positive'
  new_word = 'pos'
  # Find rows containing the target word and replace it with the new word
  df_filled["Sentiment"] = df_filled["Sentiment"].replace(old_word, new_word)

  old_word = 'Postitive'
  new_word = 'pos'
  # Find rows containing the target word and replace it with the new word
  df_filled["Sentiment"] = df_filled["Sentiment"].replace(old_word, new_word)

  old_word = 'Neutral'
  new_word = 'neu'
  # Find rows containing the target word and replace it with the new word
  df_filled["Sentiment"] = df_filled["Sentiment"].replace(old_word, new_word)

  old_word = 'Negative'
  new_word = 'neg'
  # Find rows containing the target word and replace it with the new word
  df_filled["Sentiment"] = df_filled["Sentiment"].replace(old_word, new_word)


  old_word = 'Negatve'
  new_word = 'neg'
  # Find rows containing the target word and replace it with the new word
  df_filled["Sentiment"] = df_filled["Sentiment"].replace(old_word, new_word)

  old_word = 'Neutal'
  new_word = 'neu'
  # Find rows containing the target word and replace it with the new word
  df_filled["Sentiment"] = df_filled["Sentiment"].replace(old_word, new_word)


  old_word = 'Neutural'
  new_word = 'neu'
  # Find rows containing the target word and replace it with the new word
  df_filled["Sentiment"] = df_filled["Sentiment"].replace(old_word, new_word)



  X = df_filled["Text"]
  y = df_filled["Sentiment"]

  # Convert 'X' to a NumPy array and reshape it to 2D
  X = np.array(X).reshape(-1, 1)

  # Initialize the RandomOverSampler with 'auto' sampling strategy
  oversampler = RandomOverSampler(sampling_strategy='auto')

  # Fit and transform the data to perform oversampling
  X_resampled, y_resampled = oversampler.fit_resample(X, y)

  # Create a new DataFrame with the resampled values
  df_filled = pd.DataFrame({'Text': X_resampled.ravel(), 'Sentiment': y_resampled})


  # You can apply random insertion to all rows in the 'Text' column
  #df_filled['Text'] = df_filled['Text'].apply(lambda x: random_insertion(x, n=1))

  df_filled = df_filled.rename(columns={'Text': 'text', 'Sentiment': 'label'})
  # Save the modified DataFrame back to the CSV file
  df_filled.to_csv('/content/drive/MyDrive/Tower_Sentiment_New.csv', index=False)


  print(len(df_filled))
  df_filled.head()

  data =  df_filled
  grouped_data = grouped_df
  return data, grouped_data

In [11]:
data1, grouped_data1 = cleandata(df1)
data2, grouped_data2 = cleandata(df2)
data3, grouped_data3 = cleandata(df3)
data4, grouped_data4 = cleandata(df4)
#data5, grouped_data5 = cleandata(df5)

3524
3524


<ipython-input-10-c385df99433b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filled["Sentiment"] = df_filled["Sentiment"].replace(old_word, new_word)
<ipython-input-10-c385df99433b>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filled["Sentiment"] = df_filled["Sentiment"].replace(old_word, new_word)
<ipython-input-10-c385df99433b>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

1926
3348
3348
1194
2704
2704
753
4035
4035


<ipython-input-10-c385df99433b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filled["Sentiment"] = df_filled["Sentiment"].replace(old_word, new_word)
<ipython-input-10-c385df99433b>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filled["Sentiment"] = df_filled["Sentiment"].replace(old_word, new_word)
<ipython-input-10-c385df99433b>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

771


<ipython-input-10-c385df99433b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filled["Sentiment"] = df_filled["Sentiment"].replace(old_word, new_word)
<ipython-input-10-c385df99433b>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filled["Sentiment"] = df_filled["Sentiment"].replace(old_word, new_word)
<ipython-input-10-c385df99433b>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [12]:
def cleangrouped_data(grouped_data):
  grouped_data.drop(grouped_data.columns[[0]], axis=1, inplace=True)
  grouped_data = grouped_data.dropna()
  grouped_data.head()

  old_word = 'Neural'
  new_word = 'neu'
  # Find rows containing the target word and replace it with the new word
  grouped_data['FullSentiment'] = grouped_data['FullSentiment'].replace(old_word, new_word)

  old_word = 'Positve'
  new_word = 'pos'
  # Find rows containing the target word and replace it with the new word
  grouped_data['FullSentiment'] = grouped_data['FullSentiment'].replace(old_word, new_word)
  old_word = 'positive'
  new_word = 'pos'
  # Find rows containing the target word and replace it with the new word
  grouped_data['FullSentiment'] = grouped_data['FullSentiment'].replace(old_word, new_word)

  old_word = 'Positive'
  new_word = 'pos'
  # Find rows containing the target word and replace it with the new word
  grouped_data['FullSentiment'] = grouped_data['FullSentiment'].replace(old_word, new_word)

  old_word = 'Postitive'
  new_word = 'pos'
  # Find rows containing the target word and replace it with the new word
  grouped_data['FullSentiment'] = grouped_data['FullSentiment'].replace(old_word, new_word)

  old_word = 'Neutral'
  new_word = 'neu'
  # Find rows containing the target word and replace it with the new word
  grouped_data['FullSentiment'] = grouped_data['FullSentiment'].replace(old_word, new_word)

  old_word = 'Negative'
  new_word = 'neg'
  # Find rows containing the target word and replace it with the new word
  grouped_data['FullSentiment'] = grouped_data['FullSentiment'].replace(old_word, new_word)


  old_word = 'Negatve'
  new_word = 'neg'
  # Find rows containing the target word and replace it with the new word
  grouped_data['FullSentiment'] = grouped_data['FullSentiment'].replace(old_word, new_word)

  old_word = 'Neutal'
  new_word = 'neu'
  # Find rows containing the target word and replace it with the new word
  grouped_data['FullSentiment'] = grouped_data['FullSentiment'].replace(old_word, new_word)


  old_word = 'Neutural'
  new_word = 'neu'
  # Find rows containing the target word and replace it with the new word
  grouped_data['FullSentiment'] = grouped_data['FullSentiment'].replace(old_word, new_word)

  grouped_data = grouped_data.rename(columns={'Text': 'text', 'FullSentiment': 'label'})
  return grouped_data

In [13]:
grouped_data1 = cleangrouped_data(grouped_data1)
grouped_data2 = cleangrouped_data(grouped_data2)
grouped_data3 = cleangrouped_data(grouped_data3)
grouped_data4 = cleangrouped_data(grouped_data4)

<ipython-input-12-e33aed38202d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grouped_data['FullSentiment'] = grouped_data['FullSentiment'].replace(old_word, new_word)
<ipython-input-12-e33aed38202d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grouped_data['FullSentiment'] = grouped_data['FullSentiment'].replace(old_word, new_word)
<ipython-input-12-e33aed38202d>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [14]:
def concatinateDatasets(data1, data2):
  combined_data = pd.concat([data1, data2], axis=0, ignore_index=True)

  return combined_data

def writeCsv(data):
  new_data = data.to_csv('/content/drive/MyDrive/new_data.csv', index=False)
  return new_data

In [15]:
combined1 = concatinateDatasets(data1, grouped_data1)
combined2 = concatinateDatasets(data2, grouped_data2)
combined3 = concatinateDatasets(data3, grouped_data3)
combined4 = concatinateDatasets(data4, grouped_data4)

combined1 = concatinateDatasets(combined1, combined2)
combined2 = concatinateDatasets(combined3, combined4)

combined_data = concatinateDatasets(combined1, combined2)

In [16]:
combined_data["label"] = combined_data["label"].str.lower()

In [17]:
x = combined_data["text"]
y = combined_data["label"]

In [18]:
texts =   x
sentiments = y

data = []

# Iterate through each row in the dataset
for text, sentiment in zip(texts, sentiments):
    data.append((text, sentiment))

In [19]:
sentimentArray1 = []
sentimentArray2 = []
sentimentArray3 = []
sentimentArray4 = []
sentimentArray5 = []
score = []
length = len(x)

In [21]:
for i in range (length):
    data = combined_data["text"][i]
    #print(data)
    new_text_features = tfidf_vectorizer.transform([data])
    predicted_sentiment = nb_classifier.predict(new_text_features)[0]
    #print(predicted_sentiment)
    sentimentArray1.append(predicted_sentiment)
    predicted_sentiment = logistic.predict(new_text_features)[0]
    sentimentArray2.append(predicted_sentiment)
    predicted_sentiment = decision.predict(new_text_features)[0]
    sentimentArray3.append(predicted_sentiment)
    predicted_sentiment = forest.predict(new_text_features)[0]
    sentimentArray4.append(predicted_sentiment)
    predicted_sentiment = svm.predict(new_text_features)[0]
    sentimentArray5.append(predicted_sentiment)




In [22]:


# Using a loop to compare each element
def finalSentiment(sentimentArray):
    final = []
    for i in range(len(combined_data['label'])):
        if combined_data['label'][i] == sentimentArray[i]:
            #print(f"Element {i}: Matched - combined_data_label: {combined_data['label'][i]}, sentimentarray: {sentimentArray[i]}")
            final.append(1)
        else:
            #print(f"Element {i}: Not Matched - combined_data_label: {combined_data['label'][i]}, sentimentarray: {sentimentArray[i]}")
            final.append(0)
    #print(final)  # Move this line outside of the loop
    return final

# Assuming sentimentArray1, sentimentArray2, etc., are defined elsewhere in your code
final1 = finalSentiment(sentimentArray1)
final2 = finalSentiment(sentimentArray2)
final3 = finalSentiment(sentimentArray3)
final4 = finalSentiment(sentimentArray4)
final5 = finalSentiment(sentimentArray5)


In [23]:
def accuracy(final):
  count = 0
  for i in range (len(final)):
    t = final[i]
    if t == 1:
      count += 1

  print(count)
  accuracy = (count / len(final)) * 100
  print(accuracy)

In [24]:
accuracy(final1)
accuracy(final2)
accuracy(final3)
accuracy(final4)
accuracy(final5)

2062
42.55055716054478
2445
50.45398266611638
2217
45.74907139909203
2425
50.041271151465125
2742
56.58274865868758
